## Eye Module version 0.04

In [1]:
# package inclusions
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D
from tensorflow.keras.layers import Input, Dense
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.applications.vgg16 import VGG16
import pandas as pd
import numpy as np
import os
from numpy import argmax

In [2]:
# Get location of input
PATH = "C:\\Users\\kshit\\PyPyProjects\\Capstone\\BaselineData"
data_dir_list = os.listdir(PATH)
data_dir_list

['Crossed_Eye', 'Glaucoma', 'healthy']

In [3]:
img_rows=224
img_cols=224
num_channel=3

img_data_list=[]
classes_names_list=[]
target_column=[]
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting image from {} folder".format(dataset))
    img_list= os.listdir(PATH +"/"+ dataset)
    for img in img_list:
        input_img = cv2.imread(PATH +"/"+dataset+"/"+img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

Getting image from Crossed_Eye folder
Getting image from Glaucoma folder
Getting image from healthy folder


In [4]:
num_classes = len(classes_names_list)
print("num_classes",num_classes)
img_data = np.array(img_data_list) # convert images in numpy array 
img_data = img_data.astype('float32')
img_data /= 255
print("Shape of image data",img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape 
print("number of samples",num_of_samples)

num_classes 3
Shape of image data (1028, 224, 224, 3)
number of samples 1028


In [5]:
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)
np.unique(target_column)

array([0, 1, 2], dtype=int64)

In [6]:
target_column_hotcoded = to_categorical(target_column,num_classes)
X,Y = shuffle(img_data,target_column_hotcoded,random_state=2)
X_train,X_temp,y_train,y_temp = train_test_split(X,Y,test_size=0.3,random_state=2)
X_test,X_val,y_test,y_val = train_test_split(X_temp,y_temp,test_size=0.3,random_state=2)

### Create CNN Architecture

In [7]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

In [8]:
hist = model.fit(X_train,y_train,batch_size = 40,epochs = 20,verbose=1,validation_data=(X_test,y_test))
score = model.evaluate(X_test,y_test,batch_size = 40)
print('Test Loss',score[0])
print("Test Accuracy",score[1])

Epoch 1/20
18/18 [==============================] - 119s 7s/step - loss: 2.0787 - accuracy: 0.6370 - val_loss: 0.5468 - val_accuracy: 0.8056
Epoch 2/20
18/18 [==============================] - 122s 7s/step - loss: 0.4593 - accuracy: 0.8122 - val_loss: 0.4721 - val_accuracy: 0.7963
Epoch 3/20
18/18 [==============================] - 125s 7s/step - loss: 0.4295 - accuracy: 0.8150 - val_loss: 0.4860 - val_accuracy: 0.8009
Epoch 4/20
18/18 [==============================] - 128s 7s/step - loss: 0.4183 - accuracy: 0.8192 - val_loss: 0.4795 - val_accuracy: 0.7917
Epoch 5/20
18/18 [==============================] - 125s 7s/step - loss: 0.4193 - accuracy: 0.8206 - val_loss: 0.4591 - val_accuracy: 0.8009
Epoch 6/20
18/18 [==============================] - 124s 7s/step - loss: 0.4150 - accuracy: 0.8234 - val_loss: 0.4802 - val_accuracy: 0.8009
Epoch 7/20
18/18 [==============================] - 127s 7s/step - loss: 0.4184 - accuracy: 0.8192 - val_loss: 0.4840 - val_accuracy: 0.7963
Epoch 8/20
18

### Saving this model 

In [12]:
#model.save('./models/Model_Name', save_format='tf')
model.save('eye_model_2',save_format='tf')
print("Saved model to disk")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: eye_model_2\assets
Saved model to disk


### Reloading the model

In [10]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

# load model
# model = load_model('eye_model_2.pb')
# summarize model.
# model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

In [11]:
#score = model.evaluate(X_test,y_test,batch_size = 40)
#print('Test Loss',score[0])
#print("Test Accuracy",score[1])

6/6 [==============================] - 6s 997ms/step - loss: 0.4414 - accuracy: 0.7917
Test Loss 0.44140467047691345
Test Accuracy 0.7916666865348816
